Denoise the MDT directly
====
Instead of working with currents (gradient of MDT, times some constants + coriolis parameter) we can probably smooth + denoise the MDT directly

Read in MDTs
----

In [ ]:
import pathlib

# Need to mount the SING RDSF dir somewhere
rdsf_dir = pathlib.Path("~/geog_rdsf/").expanduser()

noisy_filepath = (
    rdsf_dir
    / "data"
    / "projects"
    / "SING"
    / "richard_stuff"
    / "Table2"
    / "dtu18_eigen-6c4_do0280_rr0004.dat"
)
clean_filepath = (
    rdsf_dir / "data" / "projects" / "dtop" / "cmip6" / "cmip6_historical_mdts_yr5.dat"
)

assert noisy_filepath.exists()
assert clean_filepath.exists()

In [ ]:
import numpy as np
from current_denoising.generation import ioutils

# It's called read_currents, but actually just reads the array
noisy_mdt = ioutils.read_currents(noisy_filepath)
noisy_mdt[noisy_mdt == -1.9e19] = np.nan
clean_mdt = ioutils.read_clean_mdt(
    clean_filepath,
    clean_filepath.with_stem(clean_filepath.stem + "_meta").with_suffix(".txt"),
    year=2001,
    model="CMCC-CM2-HR4",
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import AxesImage

from current_denoising.utils import util


def mdt_imshow(current_grid: np.ndarray, axis: plt.Axes, **kwargs) -> AxesImage:
    """Imshow for MDTs - no extent set"""
    lat, long = util.lat_long_grid(current_grid.shape)
    extent = kwargs.get("extent", [long[0], long[-1], lat[0], lat[-1]])

    imshow_kw = {
        "origin": "upper",
        "cmap": "magma",
        "vmax": 2,
        "vmin": -2,
    }
    imshow_kw.update(kwargs)
    imshow_kw["extent"] = extent

    im = axis.imshow(current_grid, **imshow_kw)
    im.set_extent(extent)

    return im


fig, axes = plt.subplots(2, 1, figsize=(12, 12))

im = mdt_imshow(noisy_mdt, axes[0])
fig.colorbar(im, ax=axes[0])

im = mdt_imshow(clean_mdt, axes[1])
fig.colorbar(im, ax=axes[1])

Smooth and find residual
----
This is the "noise" we want to remove

In [ ]:
"""
Applying a Gaussian filter to the gridded field is non-trivial (the grid point size changes with latitude)
"""

from scipy.ndimage import gaussian_filter
from matplotlib.colors import LogNorm

from current_denoising.generation import mdt


# TODO for now just do a naive smoothing
def naive_smooth(img: np.ndarray) -> np.ndarray:
    """
    Invalid but simple smoothing of a gridded field containing NaNs

    Invalid since the kernel is constant in size in terms of grid points,
    which means it varies in size spatially.
    """
    nan_mask = np.isnan(img)

    field = mdt.fill_nan_with_nearest(img)

    # 8 grid points -> around 200km radius at equator
    field = gaussian_filter(field, sigma=8)
    return np.where(nan_mask, np.nan, field)


fig, axes = plt.subplots(2, 1, figsize=(12, 12))

noisy_mdt_smoothed = naive_smooth(noisy_mdt)
residual = noisy_mdt - noisy_mdt_smoothed

im = mdt_imshow(noisy_mdt_smoothed, axes[0])
axes[0].set_title("Smoothed MDT (naive)")
fig.colorbar(im, ax=axes[0])

im = mdt_imshow(residual, axes[1], vmin=-0.5, vmax=0.5)
axes[1].set_title("Residual")
fig.colorbar(im, ax=axes[1])

fig.tight_layout()

Optional: train a GAN to generate realistic-looking tiles of MDT noise
----

So that we have a large set of training data for the denoising model

Optional: stitch these tiles together
----
We want to train our denoising model on larger patches of MDT, so that it can learn the slowly-varying signal

This won't be necessary if our GAN is a TileGAN which can generate arbitrary-sized patches

Train the denoising model
----
Train the denoising model with synthetic noise + real patches of clean MDT

Evaluate the denoising model on different noisy MDTs
----